# Optimization Dataset

In [1]:
# set work dir if PyEPO is not installed
import os
os.chdir("../pkg")

PyEPO ontains synthetic data generator and a dataset class ``optDataset`` to wrap data samples.

## 1 Data Generator 

``pyepo.data`` includes synthetic datasets for three of the most classic optimization problems: the shortest path problem, the multi-dimensional knapsack problem, and the traveling salesman problem.

The synthetic datasets include features $\mathbf{x}$ and cost coefficients $\mathbf{c}$. The feature vector $\mathbf{x}_i \in \mathbb{R}^p$ follows a standard multivariate Gaussian distribution $\mathcal{N}(0, \mathbf{I})$, and the corresponding cost $\mathbf{c}_i \in \mathbb{R}^d$ comes from a polynomial function $f(\mathbf{x}_i)$ multiplicated with a random noise $\mathbf{\epsilon}_i \sim  U(1-\bar{\epsilon}, 1+\bar{\epsilon})$. The details of $f(\mathbf{x}_i)$ can be seen [here](https://khalil-research.github.io/PyEPO/build/html/content/examples/data.html).

In general, there are several parameters that users can control:
- num_data ($n$): data size
- num_features ($p$): feature dimension of costs $\mathbf{c}$
- deg ($deg$): polynomial degree of function $f(\mathbf{x}_i)$ 
- noise_width ($\bar{\epsilon}$):  noise half-width of $\mathbf{\epsilon}$
- seed: random state seed to generate data

### 1.1 Shortest Path 

$c_i^j = [\frac{1}{{3.5}^{deg}} (\frac{1}{\sqrt{p}}(\mathcal{B} x_i)_j + 3)^{deg} + 1] \cdot \epsilon_i^j$, where $\mathcal{B}$ is a random matrix.

The following code is to generate data for the shortest path on the **5x5** grid network:

In [2]:
import pyepo
grid = (5,5) # grid size
n = 1000 # number of data
p = 5 # feature dimention
deg = 4 # polynomial degree
e = 0.5 # noise half-width

Auto-Sklearn cannot be imported.


In [3]:
# generate data for grid network (features and costs)
feats, costs = pyepo.data.shortestpath.genData(num_data=n+1000, num_features=p, grid=grid,
                                               deg=deg, noise_width=e, seed=42)

In [4]:
# features
print("Features:")
print(feats[0])
# costs
print("Costs:")
print(costs[0])

Features:
[-0.68002472  0.2322537   0.29307247 -0.71435142  1.86577451]
Costs:
[0.35391761 1.07858994 1.01396442 0.66493631 0.29919599 0.19120229
 0.92962621 0.24648609 0.33950669 0.19379419 0.78927149 0.52358102
 0.66013164 1.0846318  0.73023344 0.53221019 0.34958448 0.33129203
 1.4367641  0.72844442 1.39402493 0.89432676 1.03169003 0.46667478
 0.60684515 1.7148205  1.62555298 2.13901473 0.375338   0.51937908
 1.30751427 2.39109315 0.51398154 1.02980917 0.73931099 0.23779171
 0.35521389 0.25666491 0.70956306 1.69988172]


### 1.2 Knapsack 

Because we assume that the uncertain coefficients exist only on the objective function, the weights of items are fixed throughout the data. We define the number of items as $m$ and the dimension of resources is $k$.

$c_i^j = \lceil [\frac{5}{{3.5}^{deg}} (\frac{1}{\sqrt{p}}(\mathcal{B} x_i)_j + 3)^{deg} + 1] \cdot \epsilon_i^j \rceil$, where $\mathcal{B}$ is a random matrix.

In [5]:
import pyepo
m = 32 # number of items
k = 2 # resource dimension
n = 1000 # number of data
p = 5 # feature dimention
deg = 4 # polynomial degree
e = 0.5 # noise half-width

In [6]:
# generate data for 2D kansack (weights, features and costs)
weights, feats, costs = pyepo.data.knapsack.genData(num_data=n+1000, num_features=p, num_items=m, 
                                                    dim=k, deg=deg, noise_width=e, seed=42)

In [7]:
# features
print("Weights:")
print(weights)
print("Features:")
print(feats[0])
# costs
print("Costs:")
print(costs[0])

Weights:
[[4.02 7.35 6.48 5.7  4.06 3.71 4.88 3.2  4.02 4.21 7.66 5.14 6.3  7.58
  3.87 6.72 3.99 6.59 4.51 4.3  4.49 6.08 5.57 6.43 7.91 7.13 5.93 6.85
  4.91 7.43 5.76 4.6 ]
 [7.59 6.13 3.21 5.52 5.35 6.44 3.48 7.74 3.58 4.69 7.75 4.87 7.63 5.7
  4.89 7.45 4.74 7.45 3.5  6.63 3.54 5.43 6.19 4.3  7.84 6.06 4.34 3.2
  6.28 4.66 5.73 6.87]]
Features:
[-0.60139725 -0.84661436  0.81615272  0.47170729 -0.2854893 ]
Costs:
[2. 5. 6. 4. 1. 2. 1. 4. 2. 2. 5. 5. 5. 5. 3. 2. 4. 2. 2. 2. 4. 2. 2. 4.
 3. 2. 1. 3. 4. 2. 3. 5.]


### 1.3 Traveling Salesperson

For Traveling Salesperson (TSP),  number of nodes $m$ is the addtional parameter to generate data.

The distance consists of two parts: one comes from Euclidean distance, the other derived from feature encoding. For Euclidean distance, we generate random coordinates. For feature encoding, it is $\frac{1}{{3}^{deg - 1} \sqrt{p}} ((\mathcal{B} x_i)_j + 3)^{deg} \cdot \epsilon_i^j$, where $\mathcal{B}$ is a random matrix.

In [8]:
import pyepo
m = 20 # number of items
n = 1000 # number of data
p = 5 # feature dimention
deg = 4 # polynomial degree
e = 0.5 # noise half-width

In [9]:
feats, costs = pyepo.data.tsp.genData(num_data=n+1000, num_features=p, num_nodes=m, 
                              deg=deg, noise_width=e, seed=42)

In [10]:
# features
print("Features:")
print(feats[0])
# costs
print("Costs:")
print(costs[0])

Features:
[-1.14168911 -0.19365946 -0.71682232 -1.86653662 -0.08268069]
Costs:
[ 3.7469  3.5177  3.5663  3.2863 20.8808  3.5013  4.6786  3.663   9.4076
  1.6293  8.884   2.8502  5.8908  4.8622 18.5146  5.7186  0.5103  3.1516
  8.1697  6.4878 10.9322  0.766  12.8021  7.2463 16.3108  3.0362 11.5717
  9.5445  3.901   4.9748  2.0201  2.4792  5.3137 26.4226 19.2363  2.3725
  1.9566  6.9787  6.7527  6.1478  9.8816  4.8733  4.8745  2.1084  6.8356
  7.9441  6.6348  3.2901 10.0526  4.4324  3.8492  6.6119  8.6355 22.0806
  5.5779  1.3496  8.7577  5.0559  6.8631  6.8527  3.5512  3.4764  4.0971
  4.0949  4.0859  7.7309  7.4587  1.2292 34.8099  4.9677 17.7628  3.1806
  6.3381  3.5345 18.8379  6.9102  6.5611  3.818   4.2513 16.7784  4.5815
  5.3674  3.3281 13.4974  2.304   4.8323  3.4517  7.0979  5.5551 10.3592
  3.6593 15.9906  5.1407  4.9056  2.8864  5.8821  4.7064 16.2864  8.4832
  6.8939  3.0092  6.5853  3.5005  6.8579  4.3257  2.9691  5.9071  4.8555
  4.6766  4.2926  3.0147  4.5696  1.5417  2.2

## 2 Introduction to optDataset

``optDataset`` is PyTorch Dataset, which stores the features and their corresponding costs of the objective function, and **solves optimization problems to get optimal solutions and optimal objective values**.

``optDataset`` is **not** necessary for training with PyEPO, but it can be easier to obtain optimal solutions and objective values when they are not available in the original data.

As the following example, ``optDataset`` and Pytorch ``DataLoader`` wrap the data samples, which can make the model training cleaner and more organized.

### 2.1 Generate Data 

We generate data for the shortest path on the 5x5 grid network first.

In [11]:
# grid size
grid = (5,5)

In [12]:
# generate data
num_data = 1000 # number of data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

### 2.2 Build OptModel

In order to obtain optimal solutions, ``optDataset`` requires the corresponding ``optModel`` is a module of PyEPO library, which is designed as a container for any "black box" solver. The tutorial on ``optModel`` is [here](https://github.com/khalil-research/PyEPO/blob/main/notebooks/01%20Optimization%20Model.ipynb).

Here we load the pre-defined [``pyepo.model.grb.shortestPathModel``](https://khalil-research.github.io/PyEPO/build/html/content/examples/model.html#pre-defined-models), which uses Gurobi to build a linear program.

In [ ]:
from pyepo.model.grb import shortestPathModel
# init model
optmodel = shortestPathModel(grid) 

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-09


### 2.3 Train-Test Split 

Split features and costs into random train and test subsets, where the test size is 1000.

In [ ]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)

### 2.4 Wrap Features and Costs with optDataset

``optDataset`` accepts features features $\mathbf{x}$ and cost coefficients $\mathbf{c}$, then ultilizes ``optModel`` to find optimal solutions $\mathbf{w}^*$ and objective values $\mathbf{z}^*$.

In [ ]:
# get optDataset
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

It takes time for solving optimization problem per instance.

### 2.5 Set PyTorch DataLoader

PyTorch ``DataLoader`` combines a dataset and a sampler, and provides an iterable over the given dataset. 

In [ ]:
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)